# Developing a Blood Bowl python package to plot board positions

I am a Blood Bowl scholar who like to perform Nufflytics in Python.
To do so, we need a utility package / library to work with replay data.
We also need a standard way to describe Blood Bowl games in a compact way, that is both human and machine readable.
In chess, there is the Portable Game Notation, which has become the de facto standard of describing Chess games.
I have identified the godfather of Blood Bowl Game Notation, David Morgan-Mar. Already back in 2002 he developed a notation for the purpose of sharing game logs over the internet. [https://www.dangermouse.net/games/bloodbowl/rules.html]

If we could converge on a standard Fantasy Football Game Notation, it serve would many purposes:
* It would allow us to interchange data between software
* it would help to train AI engines.
This blog post describes what I did so far. 
On the FUMBBL website, a lot of high quality replay data is available as well as an API to conveniently fetch the data.
In addition, the API provides up to date roster information.


# python-fumbbl_replays

In [ ]:
%pip install -e .

# Plotting Blood Bowl board positions

If we want to describe a board state, we need to describe the pieces, and we need to describe the location of the pieces.
(We also need to describe the "state" of the pieces, as players can be either standing, prone, or stunned, and can be in various special states such as "Bone head", "Rooted", "Hypnotized" etc. A full game state also contains additional information on rerolls, players on the bench etc. This is not yet implemented)

Let's start with the location of the pieces. A grid reference system is needed.
The game board of Blood Bowl has dimensions 15 x 26.
It has cognitive benefit to use numbers for one dimension, and letters for the other dimensions. Fancy word: alphanumeric.
Chess over the centuries has had various notations, and this notation is the one that became universally accepted.
[https://en.wikipedia.org/wiki/Algebraic_notation_(chess)]
The only choice left for us is then, which axis should have letters, and which axis should have the numbers.

A strong argument was made on the BotBowl discord that distance to the end zone is very important in BB.
By using numbers for the long axis, we can easily deduce that a Gutter Runner at position c15 is in scoring position: It needs 11 movement to score a touchdown at c26.
This notation is also used by Cow Daddy Gaming in his "What´s the play" puzzles.

I wrote a function `show_boardpos()` that displays the name of all the board positions.


In [ ]:
import fumbbl_replays as fb

fb.show_boardpos(rotation = 'H')

Next is that we need a way to describe the playing pieces, and visualize them. In chess it is easy, there are only six different ones.
In Blood Bowl, there are roughly 200 different playing pieces (30 teams, times 5 positionals, plus 50+ star players).
Here the concept of a roster can help us out. 
I wrote a function `fetch_roster()` that fetches rosters from FUMBBL and displays the positions.
It also contains links to icons that can represent the piece on the board.
Take for example the High Elf roster.

In [ ]:
roster = fb.fetch_roster("High Elf")
roster

It has four different pieces, AND FUMBBL has also already solved our problem of denoting them using text.
So if we want to describe some action involving a High Elf Catcher, and there are four of them on the board, we could denote them by C1, C2, C3 and C4.
This is compact, and has meaning within the context of the High Elf roster.

If we combine the descriptions of the pieces, and their location, we have enough to describe for example an initial setup formation before kick-off.

In [ ]:
roster = fb.fetch_roster("High Elf")

my_setup = ['setup', ['L1: g13', 'L2: h13', 'L3: i13', 'Z1: c11', 'Z2: m11', 'T1: h6', 'L4: e11', 
                      'L5: k11', 'C1: l10', 'C2: d10', 'L6: h11']]

I wrote a function `create_position()` that combines the roster and the setup annotation to create an object that contains all the information to make a nice plot of the board state. The function `print_position()` prints a nicely formatted summary of the position.
As default, a position is created for the home team, denoted as "teamHome".

In [ ]:
positions = fb.create_position(roster, my_setup)
fb.print_position(positions)

Let's suppose that the High Elf team is playing Gnomes, for which we also fetch the roster and create a board position.
As we already have a home team, we refer to this team as "teamAway".

In [ ]:
roster = fb.fetch_roster("Gnome")

my_setup = ['setup', ['T2: j14', 'T1: f14', 'F1: h20', 'I1: b14', 'I2: n14', 'L3: e14', 'L6: k14', 
                      'B2: m15', 'B1: c15', 'L4: g15', 'F2: i16']]

positions2 = fb.create_position(roster, my_setup, 'teamAway')

fb.print_position(positions2)

As a final step before plotting, we add both positions together.
As both are `pandas` DataFrames, we use the `concat()` function from `pandas`  to combine ("concatenate") them.

In [ ]:
import pandas as pd

positions = pd.concat([positions, positions2])

The function `create_plot()` plots the board position.
By default, it plots a horizontal pitch, with the team denoted as "teamHome" in red, and the other team in blue.

In [ ]:
fb.create_plot(positions)

The `create_plot()` function allows us the swap the color of the teams, to change the pitch orientation to vertical, and to add a layer of semi-transparant tacklezones.

In [ ]:
fb.create_plot(positions, red_team = "teamAway", orientation = 'V', tackle_zones = True)

The library also support moving single pieces (players). It currently only works for pieces that already exist in a board position.
In the plot above, suppose we want to move the Woodland Fox F1 to board position `o26`:

In [ ]:
positions = fb.move_piece(positions, "teamAway", "F1", "o26")

fb.create_plot(positions, red_team = "teamAway")

# Plotting board positions from FUMBBL replays

Above we created board positions from scratch, using rosters from FUMBBL and a simple way to describe a board position.
The package also allows us to plot board positions extracted from FUMBBL replay files.
At this moment, only the board position right before kick-off can be plotted.
Suppose we want to plot this position for match 4551601.

We first need to fetch the replay data. The `fetch_data()` function takes the match_id as argument and returns five objects:
the match_id, replay_id, a positions object, which team is the receiving team (i.e. playing offense), and some metadata (coach names, race names, and match touchdown result).

In [ ]:
match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id = 4551601)

To plot the board state right before kick-off, we can use the `create_plot()` function in the same way as above.
We plot the receiving team in red so we can see which team is playing offense and which team is playing defense.

In [ ]:
fb.create_plot(positions, red_team = receiving_team)

Adjusting this board position both moving players one-by-one works also in the same way as above.

In [ ]:
positions = fb.move_piece(positions, "teamAway", "Z1", "b26")
positions = fb.move_piece(positions, "teamAway", "Z2", "o26")

fb.create_plot(positions, red_team = receiving_team)

Suppose we think that the offensive setup is awesome, and we wish to share this setup with other coaches.
Here the compact way to describe a setup using player abbreviations and the alphanumeric grid system comes in handy.
To get the position in this notation I wrote the function `get_position()`.


In [ ]:
fb.get_position(positions, home_away = 'teamHome')


Suppose we think this setup is awesome, but it would be even better if the illusionist in row `l` would be a second Woodland Fox.
We can take the setup (copy-paste), change the setup slightly, and create a new position.
As we now only have a single team, we can rotate the pitch and crop to show only the upper part of it.

In [ ]:
roster = fb.fetch_roster("Gnome")

my_setup = ['setup', ['T2: j13', 'T1: f13', 'F2: h8', 'B2: g13', 'B1: i13', 'L2: h13', 'L5: m13', 'L4: k13', 'F2: l13', 'I1: b12', 'L1: k12']]

positions = fb.create_position(roster, my_setup)

fb.create_plot(positions, orientation= "V", crop = "upper")

# Application 1: Visualizing a particular setup

On FUMBBL there is a great guide on defensive setups. Suppose we wish to visualize the "arrowhead" setup, advised for Undead.

_The arrowhead defense is a good defense for Undead, Necromantic, or Chaos Renegades, against a highly mobile opponent. It’s similar to a ziggurat or chevron defense, but the 3-column midfielders or safeties have been moved into the 0-column to prevent runs up the gut, and also to be able to redeploy from a central position, while the 2-column strong midfielders have been pushed out to the 3-column to form a spine screen. It’s strong in the wide zone and up the center, but the 3-column midfielders are seriously exposed, and unless they have both a lot of Strength and the Stand Firm skill, this position is highly vulnerable._

The Arrowhead Defense

```
7 6 5 4|3 2 1 0 1 2 3|4 5 6 7   column

- - - -|- - x x x - -|- - - -   Line

- - - -|- - - - - - -|- - - -   -1

- - x -|x - - x - - x|- x - -   -2

- x - -|- - - x - - -|- - x -   -3
```

In [ ]:
fb.show_boardpos(rotation = 'H')

In [ ]:
roster = fb.fetch_roster("Shambling Undead")
roster

In [ ]:
my_setup = ['setup', ['Z1: g14', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16', 'G1: h16', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]


positions = fb.create_position(roster, my_setup)

fb.create_plot(positions, receiving_team = "teamAway", orientation = 'V', crop = "lower")

for whats the play we need to display prone and stunned as well. Should include this in the notation. Maybe x and /

my_setup = ['setup', ['Z1: g14x', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16/', 'G1: h16/', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]

# Application 2: saving defensive setups as PNGs for use in playbooks

We want to plot all defensive setups together with match outcome from the Tilean Team Cup.
The Tilean Team Cup was an online NAF tournament held on FUMBBL from march 2023 to may 2023.

In [ ]:
# point this to the location of the HDF5 datasets
path_to_datasets = '../fumbbl_datasets/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')


tilean_replays = df_matches['match_id'].values

tilean_replays = tilean_replays[0:3]
tilean_replays

In [ ]:
fullrun = 0

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id)  # gnome 4543329 #4528210 #4542768
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = True)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[2])
        race_offense.append(metadata[3])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
df_replays

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/wood_elf/1606445_4447434_kickoff_lower_defense.png")

# Application 3: Working with replays directly: FFGN


In [ ]:
my_replay = fb.fetch_replay(1602344)

df = fb.parse_replay(my_replay) 

positions = df.query('turnNr == 0 & turnMode == "setup" & Half == 1 & \
                     modelChangeId == "fieldModelSetPlayerCoordinate"').groupby('modelChangeKey').tail(1)

positions

In [ ]:
df_positions = fb.extract_rosters_from_replay(my_replay)

receiving_team = fb.determine_receiving_team_at_start(df)

In [ ]:
df = fb.fumbbl2ffgn(match_id = 4447439)

In [ ]:
pd.set_option('display.max_colwidth', None)

# Turn 1 for the offensive
(df
 .query("Half == 1 & turnNr == 1 & commandNr > 88 & commandNr < 211")
 .filter(['commandNr', 'turnMode', 'modelChangeId', 'modelChangeKey', 'modelChangeValue'])
)

In [ ]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
